In [1]:
import pickle
import torch
import matplotlib.pyplot as plt

{0: array(['brigand'], dtype='<U22'), 1: array(['correspondents'], dtype='<U22'), 2: array(['reacquired'], dtype='<U22')}


In [ ]:
# Load the data about the clusters
_, data = pickle.load(open("./glove/kmeans_clusters_500.pkl", "rb"))

# Check if GPU(s) are available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X, labels = None, None
for key in data.keys():
    if X is None:
        X = torch.Tensor(data[key])
        labels = torch.reshape(torch.Tensor([key] * len(data[key])), (-1, 1))
    else:
        X = torch.cat([X, torch.Tensor(data[key])])
        temp = torch.reshape(torch.Tensor([key] * len(data[key])), (-1, 1))
        labels = torch.cat([labels, temp])

# Create the set of unique labels
targets = [x.item() for x in labels]
unique_labels = list(set(targets))

for j in unique_labels:
    glove_vectors = []
    model_vectors = []  
    explanations = pickle.load(open(f"closest_words-{j}.pkl", "rb"))
    for exp in explanations:
        glove_vectors.append(exp["glove vector"])
        model_vectors.append(exp["produced vector"])
        
    # Model vectors is a k length list of d dimensional vectors
    # Turn it into a k x d tensor
    model_vectors = torch.Tensor(model_vectors)
    glove_vectors = torch.Tensor(glove_vectors)
    
    y = model_vectors
    model_vector_projection_matrix = y @ (y.T @ y).inverse() @ y.T
    
    y = glove_vectors
    glove_vector_projection_matrix = y @ (y.T @ y).inverse() @ y.T
    
    # Project X
    X_model = model_vector_projection_matrix @ X.T
    X_glove = glove_vector_projection_matrix @ X.T
    
    X_model = X_model.T
    X_glove = X_glove.T
    
    # Plot X_model according to the model vectors
    plt.scatter(X_model[:, 0], X_model[:, 1], c=labels, cmap='viridis')
    plt.title(f"Model vectors for cluster {j}")
    
    plt.savefig(f"model_vectors-{j}.png")
    plt.show()
    
    # Plot X_glove according to the glove vectors
    plt.scatter(X_glove[:, 0], X_glove[:, 1], c=labels, cmap='viridis')
    plt.title(f"Glove vectors for cluster {j}")
    
    plt.savefig(f"glove_vectors-{j}.png")
    plt.show()